### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")



### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}&fq={filter_query}&sort={sort}&fl={field_list}"

In [4]:
query_url

'https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=kdzQIYFomrXinIkI4GBAfbr1P6Fv6k0O&begin_date=20130101&end_date=20230531&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count'

In [52]:
# Create an empty list to store the reviews
reviews_list = []


# loop through pages 0-19
for page in range(0, 20):
    # API results show 10 articles at a time 
    query_url_pages = f"{query_url}&page={page}"

    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_url_pages).json()
 
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f"page {page}")
    except:
        print(f"No results for page: {page}")
        break

page 0
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19


In [5]:
# Preview the first 5 results in JSON format
for review in reviews_list[:5]:

# Use json.dumps with argument indent=4 to format data
    print(json.dumps(review, indent=4))

NameError: name 'reviews_list' is not defined

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
all_results_df = pd.json_normalize(reviews_list)
all_results_df

NameError: name 'reviews_list' is not defined

In [7]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early


all_results_df['title'] = all_results_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")]
)


NameError: name 'all_results_df' is not defined

In [56]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string


In [8]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = all_results_df['title'].to_list()

print(titles_list)


NameError: name 'all_results_df' is not defined

### Access The Movie Database API

In [9]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

query_url1 = f"{url}api-key={tmdb_api_key}"

print(query_url1)

https://api.themoviedb.org/3/search/movie?query=api-key=a17a0af46e8ccfc6d14a4811bf4129bb


In [10]:
tmdb_api_key

'a17a0af46e8ccfc6d14a4811bf4129bb'

In [11]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple of 50 requests
request_counter = 1
multiple = 50

# Loop through the requests
for titles in range(request_counter): 
    try:
        # Make the request here
        response = requests.get(query_url)  
        response.raise_for_status()  # Check if the request was successful
        tmdb_movies_list.append(response.json())  # Assuming response is in JSON format

    except requests.RequestException as e:
        print(f"Request failed: {e}")

    # Update the request counter
    request_counter += 1

    # Check if we need to sleep before making the next request
    # Sleep for 1 second after every multiple of 50 requests
    if request_counter % multiple == 0:
        time.sleep(1)  # Sleep for 1 second



In [17]:
 # Perform a "GET" request for The Movie Database
response = requests.get(query_url1)
response


# Include a try clause to search for the full movie details and use the except clause to print out a statement if a movie is not found
try:
    # Get movie id
    movie_id = 123  # Replace with the actual movie id
    
    # Make a request for the full movie details
    query_url1 = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
    
    # Execute "GET" request with url
    response_data = requests.get(query_url1)
    
    # Extract the genre names into a list
    genres = ["Action", "Adventure"]  # Replace with actual genre names
    
    # Extract the spoken_languages' English name into a list
    spoken_languages = ["English", "French"]  # Replace with actual spoken languages
    
    # Extract the production_countries' name into a list
    production_countries = ["USA", "UK"]  # Replace with actual production countries
    
    # Add the relevant data to a dictionary and append it to the tmdb_movies_list list
    result = {
       "title": response_data.get('title'),
        "original_title": response_data.get('original_title'),
        "budget": response_data.get('budget'),
        "original_language": response_data.get('original_language'),
        "homepage": response_data.get('homepage'),
        "overview": response_data.get('overview'),
        "popularity": response_data.get('popularity'),
        "runtime": response_data.get('runtime'),
        "revenue": response_data.get('revenue'),
        "release_date": response_data.get('release_date'),
        "vote_average": response_data.get('vote_average'),
        "vote_count": response_data.get('vote_count'),
        "genres": genres,
        "spoken_languages": spoken_languages,
        "production_countries": production_countries
    }
        
    tmdb_movies_list.append(result)
    
    # Print out the title that was found
    print(f"Movie found: {result['title']}")

except:
    print("Movie not found")



Movie not found


In [18]:
# Preview the first 5 results in JSON format
for titles in tmdb_movies_list[:5]:

# Use json.dumps with argument indent=4 to format data
    print(json.dumps(review, indent=4))

NameError: name 'review' is not defined

In [19]:
# Convert the results to a DataFrame
df_tmdb_movies = pd.DataFrame(tmdb_movies_list)

# Display the DataFrame
print(df_tmdb_movies)

  status                                          copyright  \
0     OK  Copyright (c) 2024 The New York Times Company....   

                                            response  
0  {'docs': [{'web_url': 'https://www.nytimes.com...  


### Merge and Clean the Data for Export

In [20]:
# Merge the New York Times reviews and TMDB DataFrames on title

# Merge the New York Times reviews and TMDB DataFrames on the "title" column
merged_df = pd.merge(all_results_df, df_tmdb_movies, on="title", how="inner")

# Display the merged DataFrame
print(merged_df)

NameError: name 'all_results_df' is not defined

In [21]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']


# Create a list of characters to remove
characters_to_remove = ['[', ']', "'"]


# Loop through the list of columns to fix
for col in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[col] = merged_df[col].astype(str)
    
    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[col] = merged_df[col].str.replace(char, '')

# Display the fixed DataFrame
print(merged_df)

NameError: name 'merged_df' is not defined

In [66]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=["byline.person"])
print(merged_df)

NameError: name 'merged_df' is not defined

In [ ]:
# Delete duplicate rows and reset index

merged_df = merged_df.drop_duplicates()

# Reset the index
merged_df = merged_df.reset_index(drop=True)

print(merged_df)

NameError: name 'merged_df' is not defined

In [ ]:
# Export data to CSV without the index
merged_df.to_csv("merged_data.csv", index=False)
print("Data successfully exported to CSV without the index.")


NameError: name 'merged_df' is not defined